# 1. Environment Setup

To begin, I’ll import all the essential libraries: pandas, NumPy, datetime, Plotly Express, and Plotly Graph Objects.

In [4]:
import numpy as np
import pandas as pd 
import datetime as dt 
import plotly.express as px 
import plotly.graph_objects as go

# 2. Checking and Preparing the Data¶

Now we are all set to have a first look at our data.



In [5]:
retail = pd.read_csv(r"C:\Users\sande\OneDrive\Desktop\projetcs\Customer Predictive Value Modelling\online_retail_II.csv")


In [6]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


Initial Observations

1. The 'InvoiceDate' column contains date and time information but is stored as an object, so it needs to be converted into a proper datetime format. I will create a new column, 'InvoiceDate_DT', for this conversion.

2. Most columns contain around 1,067,000 rows, except 'Description' and 'Customer ID', which have missing values. The description field is not essential for this analysis, but 'Customer ID' is critical, so rows without a customer ID will be removed.

3. The 'Customer ID' column is currently stored as float64, although int64 is more appropriate for identification numbers. I will convert its data type accordingly.

4. Since the dataset includes both Quantity and UnitPrice, I will compute a new column representing the total monetary value of each transaction.

5. To ensure proper chronological analysis, the dataset will be sorted by invoice date after preprocessing.

These preprocessing steps are essential for maintaining clean data and preparing it for customer value modelling.

In [8]:
retail['InvoiceDate_DT'] = pd.to_datetime(retail['InvoiceDate'])
retail = retail[retail['Customer ID'].notna()]
retail['Customer ID'] = retail['Customer ID'].astype(np.int64)
retail = retail.sort_values('InvoiceDate_DT')
retail['Total'] = retail['Quantity'] * retail['Price']

# 3. General Sales Analytics
 
### Most and Least Expensive Product
As a first step, I am interested in the products having highest and lowest price.

In [9]:
most_expensive = retail.loc[retail['Price'] == retail['Price'].max()]
most_expensive

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceDate_DT,Total
748142,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.0,15098,United Kingdom,2011-06-10 15:31:00,-38970.0


In [10]:
least_expensive = retail.loc[retail['Price'] == retail['Price'].min()]
least_expensive = least_expensive.groupby(['Description', 'Price'], as_index=False)['Quantity'].agg('sum')
least_expensive 

,Description,Price,Quantity
0,FLAMINGO LIGHTS,0.0,24
1,OVAL WALL MIRROR DIAMANTE,0.0,1
2,36 FOIL STAR CAKE CASES,0.0,144
3,6 RIBBONS EMPIRE,0.0,12
4,ADVENT CALENDAR GINGHAM SACK,0.0,4
...,...,...,...
56,SET/5 RED SPOTTY LID GLASS BOWLS,0.0,2
57,TV DINNER TRAY DOLLY GIRL,0.0,9
58,This is a test product.,0.0,10
59,VINTAGE CREAM CAT FOOD CONTAINER,0.0,1


- The most expensive item in the dataset has an unusual description (“Manual”), is priced at £38,970, and even has a quantity of -1, suggesting it may be a return or an administrative entry rather than a real product.

- For the least expensive items, there are 61 products priced at £0.0. Since zero-priced items usually represent errors, free samples, or adjustments, it makes more sense to exclude them to get meaningful insights into the actual minimum product price.

Now, what if we disregard zero-priced products in order to get more meaningful insights?

In [11]:
retail_non_zero_price = retail.loc[retail['Price'] != 0]
least_expensive_not_null = retail_non_zero_price.loc[retail_non_zero_price['Price'] == retail_non_zero_price['Price'].min()]
least_expensive_not_null = least_expensive_not_null.groupby(['Description', 'Price'], as_index=False)['Quantity'].agg('sum')
least_expensive_not_null

,Description,Price,Quantity
0,Bank Charges,0.001,1
1,PADS TO MATCH ALL CUSHIONS,0.001,17


- The lowest-priced actual product is “PADS TO MATCH ALL CUSHIONS”, costing only £0.001 — essentially 100 units for a penny.

- Interestingly, bank charges in the dataset are also listed at £0.001.

# 4. Customers Analysis: Total Purchase Amount

Next, I'm interested in finding out, which 10 customers have spent most in the online shop for the two years, and thus are the most valuable clients. Also, I'll define the 10 customers spending least.

In [12]:
retail_customers = retail.groupby(['Customer ID', 'Country'], as_index=False)['Total'].agg('sum')
retail_customers_asc = retail_customers.sort_values('Total').head(10)
retail_customers_desc = retail_customers.sort_values('Total', ascending=False).head(10)

In [13]:
fig = go.Figure(data=[
    go.Bar(name='Customers With Max Total Purchase Amount', 
           x=retail_customers_desc['Customer ID'].astype(str), 
           y=retail_customers_desc['Total'],
           marker_opacity=1,
           hovertext=retail_customers_desc['Country'],
           marker={'color': retail_customers_desc['Total'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_traces(texttemplate='£%{y:.3s}', textposition='inside')

fig.update_layout(title='Customers With Max Total Purchase Amount',
                  title_x=0.45,
                  xaxis_title="Customer ID",
                  yaxis_title="Total Amount, £",
                  plot_bgcolor='white')
fig.show()


In [14]:
fig = go.Figure(data=[
    go.Bar(name='Customers With Min Total Purchase Amount', 
           x=retail_customers_asc['Customer ID'].astype(str), 
           y=retail_customers_asc['Total'],
           marker_opacity=1,
           hovertext=retail_customers_desc['Country'],
           marker={'color': retail_customers_desc['Total'],
                   'colorscale': 'Rainbow'}
          )
    ])
fig.update_xaxes(side="top")

fig.update_traces(texttemplate='£%{y:.3s}', textposition='inside')

fig.update_layout(title='Customers With Min Total Purchase Amount',
                  title_x=0.45,
                  title_y=0.97,
                  xaxis_title="Customer ID",
                  yaxis_title="Total Amount Returned, £",
                  plot_bgcolor='white')
fig.show()

### What have we learned from the charts?


- The charts show that Customer 18102 is the highest spender in our online store over the two-year period, with a total purchase value of £598k, and this customer is based in the United Kingdom. A customer from the Netherlands follows closely with £523k in total spending. Overall, the top 10 highest-spending customers come from the UK, EIRE, the Netherlands, and Australia all regions associated with mature, high-income markets.

- In contrast, the chart of customers with the lowest total spending contains only negative values, which indicates these transactions were returns rather than purchases. The largest total return is £25k, again from a customer in the UK. Interestingly, the countries that appear among the highest spenders are the same ones that show the largest return amounts, likely reflecting higher transaction volumes in these regions.

# 5. Countries Analysis: Total Purchase Amount¶
Here, I will find 10 countries, total sum of orders from which was the largest, and also 10 countries, where total purchases were minimal.

In [15]:
retail_countries_purchase = retail.groupby(['Country'], as_index=False)['Total'].agg('sum')
retail_countries_purchase_asc = retail_countries_purchase.sort_values('Total').head(10)
retail_countries_purchase_desc = retail_countries_purchase.sort_values('Total', ascending=False).head(10)

In [16]:
fig = go.Figure(data=[
    go.Bar(name='Countries With Max Total Purchase Amount', 
           x=retail_countries_purchase_desc['Country'].astype(str), 
           y=retail_countries_purchase_desc['Total'],
           marker_opacity=1,
           marker={'color': retail_countries_purchase_desc['Total'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_traces(texttemplate='£%{y:.3s}', textposition='outside')

fig.update_layout(title='Countries With Max Total Purchase Sum',
                  title_x=0.45,
                  xaxis_title="Countries",
                  yaxis_title="Total Amount, £",
                  plot_bgcolor='white')
fig.show()

In [17]:
fig = go.Figure(data=[
    go.Bar(name='Countries With Min Total Purchase Amount', 
           x=retail_countries_purchase_asc['Country'].astype(str), 
           y=retail_countries_purchase_asc['Total'],
           marker_opacity=1,
           marker={'color': retail_countries_purchase_desc['Total'],
                   'colorscale': 'Rainbow'}
          )
    ])

fig.update_traces(texttemplate='£%{y:.3s}', textposition='inside')

fig.update_layout(title='Countries With Min Total Purchase Amount',
                  title_x=0.45,
                  title_y=0.97,
                  xaxis_title="Countries",
                  yaxis_title="Total Amount, £",
                  plot_bgcolor='white')
fig.show()

- When examining the top 10 countries by total purchase value, it is immediately evident that the United Kingdom dominates with a total spending of £13.8M. The next highest is EIRE, with a purchase total of £579k, which is 23 times lower than the UK’s figure. Another notable observation is that all of the top 10 countries are European, indicating that the strongest customer base for this store is concentrated within Europe.

- On the opposite end, the country with the lowest total purchase value is Saudi Arabia, whose customers spent only £131 over the two-year period. While most low-spending countries are located outside of Europe, there is one interesting exception—the Czech Republic. With additional data, it would be valuable to explore the underlying reasons behind this unusually low spending pattern.

# 6. Countries Analysis: Unique Customers Number
Next, I'm interested in which 10 countries number of unique customers is largest, and in which it is small.

In [18]:
retail_countries_customers = retail.groupby(['Country'], as_index=False)['Customer ID'].agg({'Customer ID': 'nunique'})
retail_countries_customers.rename(columns={'Customer ID': 'Customer Count'}, inplace=True)
retail_countries_customers_asc = retail_countries_customers.sort_values('Customer Count').head(10)
retail_countries_customers_desc = retail_countries_customers.sort_values('Customer Count', ascending=False).head(10)

C:\Users\sande\AppData\Local\Temp\ipykernel_2364\602919922.py:1: FutureWarning:

Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.



In [19]:
fig = go.Figure(data=[
    go.Bar(name='Countries with Largest Number of Unique Customers', 
           x=retail_countries_customers_desc['Country'].astype(str), 
           y=retail_countries_customers_desc['Customer Count'],
           marker_opacity=1,
           marker={'color': retail_countries_customers_desc['Customer Count'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_traces(texttemplate='%{y:.3s}', textposition='outside')

fig.update_layout(title='Countries with Largest Number of Unique Customers',
                  title_x=0.45,
                  xaxis_title="Countries",
                  yaxis_title="Number of Unique Customers",
                  plot_bgcolor='white')
fig.show()

In [20]:
fig = go.Figure(data=[
    go.Bar(name='Countries with Smallest Number of Unique Customers', 
           x=retail_countries_customers_asc['Country'].astype(str), 
           y=retail_countries_customers_asc['Customer Count'],
           marker_opacity=1,
           marker={'color': retail_countries_customers_desc['Customer Count'],
                   'colorscale': 'Rainbow'}
          )
    ])

fig.update_traces(texttemplate='%{y:.3s}', textposition='outside')

fig.update_layout(title='Countries with Smallest Number of Unique Customers',
                  title_x=0.45,
                  title_y=0.97,
                  xaxis_title="Countries",
                  yaxis_title="Number of Unique Customers",
                  plot_bgcolor='white')
fig.show()

- In the United Kingdom, approximately 5.41k unique customers made purchases over the two-year period, while Germany and France follow far behind with only 107 and 95 unique customers respectively. The gap between the UK and the next largest markets is extremely large, mirroring the pattern we observed earlier when analyzing total purchase amounts by country.

- We also see several countries with only a single unique customer, including Thailand, Nigeria, Iceland, Saudi Arabia, and others. Interestingly, the Czech Republic appears again as an outlier in this group of mainly Asian, South American, and African countries. Additionally, both Lithuania and Iceland also have just one customer. While Iceland’s small population could explain its low customer count, it would be valuable to explore why the Czech Republic and Lithuania show similarly low engagement.

# 7. Products Analysis: Quantity
Next, I will find 10 bestselling products and 10 products which have top number of returns.

I will remove products with description 'Discount' and 'CRUK Commission' to concentrate on the representative data.

In [21]:
retail_products = retail.groupby(['Description'], as_index=False)['Quantity'].agg('sum')
retail_products = retail_products.loc[~retail_products['Description'].isin(['CRUK Commission', 'Discount'])]
retail_products_asc = retail_products.sort_values('Quantity').head(10)
retail_products_asc['Quantity'] = retail_products_asc['Quantity'] * (-1)
retail_products_desc = retail_products.sort_values('Quantity', ascending=False).head(10)

In [22]:
fig = go.Figure(data=[
    go.Bar(name='10 Bestselling Products', 
           x=retail_products_desc['Description'].astype(str), 
           y=retail_products_desc['Quantity'],
           marker_opacity=1,
           marker={'color': retail_products_desc['Quantity'],
                   'colorscale': 'Rainbow'})
    ])

fig.update_traces(texttemplate='%{y:.3s}', textposition='outside')
fig.update_layout(barmode='group', showlegend=False)

fig.update_layout(title='10 Bestselling Products',
                  title_x=0.45,
                  xaxis_title="Products",
                  yaxis_title="Total Quantity Sold",
                  plot_bgcolor='white')
fig.show()

In [23]:
fig = go.Figure(data=[
    go.Bar(name='10 Products With Most Returns', 
           x=retail_products_asc['Description'].astype(str), 
           y=retail_products_asc['Quantity'],
           marker_opacity=1,
           marker={'color': retail_products_desc['Quantity'],
                   'colorscale': 'Rainbow'}
          )
    ])

fig.update_traces(texttemplate='%{y:.3s}', textposition='outside')
fig.update_layout(barmode='group', showlegend=False)

fig.update_layout(title='10 Products With Most Returns',
                  title_x=0.45,
                  title_y=0.97,
                  xaxis_title="Products",
                  yaxis_title="Total Quantity Returned",
                  plot_bgcolor='white')
fig.show()

- The data shows that World War II Gliders are the clear bestseller, with 107k units sold over the two-year period. Interestingly, several practical, everyday items also perform extremely well—most of the top-selling products are various holders, cake cases, along with a purse and a jumbo bag, indicating strong demand for both novelty and utilitarian goods.

- On the returns side, the product with the highest number of returns is White Cherry Lights, with 105 units returned. It is notable that many of the most frequently returned items are decorative or gift-related products, such as candles, flasks, pots, magnets, and lights. This suggests that visually oriented or seasonal items may have higher return rates, possibly due to customer expectations not matching the actual product.

# 8. Online Sales on Timeline
Finally, I will built two more charts to show how the sales amount was changing with time. I will break down data by quarters and by months.

1. Quarterly Sales

In [31]:
retail_quarters_by_sales = retail.groupby(['InvoiceQuarterID'], as_index=False)['Total'].agg('sum')
retail_quarters_by_sales['InvoiceQuarter'] = retail_quarters_by_sales['InvoiceQuarterID'].map(dict(zip(quarters_map.values(), 

quarters_map.keys())))

In [33]:
fig = go.Figure(data=[
    go.Bar(name='Quarterly Sales', 
           x=retail_quarters_by_sales['InvoiceQuarter'],
           y=retail_quarters_by_sales['Total'],
           marker_opacity=1,
           text='Total',
           marker={'color': retail_quarters_by_sales['Total'],
                   'colorscale': 'Portland'}
          )
    ])

fig.update_traces(texttemplate='%{text}: £%{y:.3s}', textposition='outside')
fig.update_layout(barmode='group', showlegend=False)

fig.update_layout(title="Quarterly Sales",
                  title_x = 0.45,
                  xaxis_title="Quarters",
                  yaxis_title="Total Amount, £",
                  plot_bgcolor='white')
fig.show()

We can easily notice the seasonal fluctuations here: people tend to buy a lot before Christmas, and afterwards, a significant drop happens.

2. Monthly Sales

In [34]:
retail_monthly = retail.copy()
retail_monthly['InvoiceMonth'] = retail_monthly['InvoiceDate_DT'].dt.to_period('M')
retail_monthly_by_sales = retail_monthly.groupby(['InvoiceMonth'], as_index=False)['Total'].agg('sum')

In [35]:
fig = go.Figure(data=[
    go.Bar(name='Monthly Sales', 
           x=retail_monthly_by_sales['InvoiceMonth'].astype(str),
           y=retail_monthly_by_sales['Total'],
           marker_opacity=1,
           marker={'color': retail_monthly_by_sales['Total'],
                   'colorscale': 'Portland'}
          )
    ])

fig.update_layout(barmode='group', showlegend=False)

fig.update_layout(title="Monthly Sales",
                  title_x = 0.45,
                  xaxis_title="Months",
                  yaxis_title="Total Amount, £",
                  plot_bgcolor='white')
fig.show()

Also with a more precise breakdown you can easily detect the Christams peaks. Note, however, how different December 2011 is from December 2010! December sales have dropped more than twice!

# 10. Conclusion

We’ve uncovered several meaningful insights throughout this analysis, and here is a concise summary of the most important findings:

- The most expensive product in the dataset is priced at £38,970, while the cheapest item costs only £0.001.

- The highest-value customer is Customer 18102 from the United Kingdom, who spent £598k over two years. In contrast, another UK customer recorded £25k in returns, representing the lowest (negative) purchase value.

- The United Kingdom leads all countries with £13.8M in total purchases and the highest number of unique customers. At the opposite end, Saudi Arabia shows the lowest purchase value. Interestingly, Czech Republic stands out as a European country with unusually low spending, which could be worth exploring from a marketing perspective.

- Among the top-selling items, practical everyday goods such as holders and cases dominate the list. However, the overall bestseller is “World War II Gliders,” likely a model airplane kit. Decorative or gift items, on the other hand, show the highest return rates.

- Customer retention dropped sharply in 2010, mirroring a decline in the average number of products sold, but both metrics improved again in 2011. With additional data, investigating the reasons behind these fluctuations would be insightful.

- Sales exhibit clear seasonality, with December 2011 generating roughly half the income of December 2010. This pattern highlights a valuable starting point for deeper seasonal trend analysis.